In [2]:
import requests
import boto3
import json
import time
API_KEY = 'RGAPI-a6773509-0116-4fdf-a293-b3224cd25cbd'

BASE_URL = "https://br1.api.riotgames.com"


def get_live_match_puuids(summoner_puuid, region='br1'):
    url = f"{BASE_URL}/lol/spectator/v5/active-games/by-summoner/{summoner_puuid}?api_key={API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        match_data = response.json()
        players = match_data['participants']
        puuids = [player['puuid'] for player in players]
        return puuids, players
    else:
        print(f"Erro ao buscar partida ao vivo: {response.status_code}")
        return None, None


def get_match_history(puuid, region='americas'):
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        match_ids = response.json()  # Retorna IDs das partidas
        match_details = []
        
       
        for match_id in match_ids:
            print("pegando detalhe da partida")
            match_detail = get_match_detail(match_id, puuid, region)
            if match_detail:
                match_details.append(match_detail)
        
        return match_details  # Retorna detalhes completos das partidas
    else:
        print(f"Erro ao buscar histórico de partidas: {response.status_code}")
        return None

# Função para obter detalhes completos de uma partida
def get_match_detail(match_id, puuid, region='americas'):
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        response = response.json()
        for i, player in enumerate(response['info']['participants']):
            if player['puuid'] == puuid:
                return response['info']['participants'][i]
        return None
    else:
        print(f"Erro ao buscar detalhes da partida {match_id}: {response.status_code}")
        return None
    
def save_to_s3(data, bucket_name, file_path):
    s3 = boto3.client('s3')
    s3.put_object(Bucket=bucket_name, Key=file_path, Body=json.dumps(data))
    print(f"Dados salvos em: s3://{bucket_name}/{file_path}")

# Função principal
def main(summoner_name):
    bucket_name = 'partida-para-prever'
    team_blue = []
    team_red = []
    
    # Obter puuids dos jogadores de uma partida ao vivo
    puuids, players = get_live_match_puuids(summoner_name)
    print("Vou pegar o histórico")
    
    if puuids:
        teste = True
        for i, puuid in enumerate(puuids): 
            if i == 3 or i == 7:
                print("esperando para não dar time out...")
                time.sleep(120)   
            match_history = get_match_history(puuid)
            
            if match_history:
                player_data = {
                    "teamId": players[i]['teamId'],
                    "matchHistory": match_history
                }
                
                # Separar por time
                if players[i]['teamId'] == 100:
                    print("time 100")
                    team_blue.append(player_data)
                else:
                    print("time 200")
                    team_red.append(player_data)
        
        # Salvar dados dos times no S3
        save_to_s3(team_blue, bucket_name, 'team_100/match_history.json')
        save_to_s3(team_red, bucket_name, 'team_200/match_history.json')



In [3]:
main('CdbJM3komApEjQd155Kxn3nXPLGbKigfnVKtAwjkRoIhX2o5Km5cfs8UQNXbFzsF3XEixjJQcuY7MA') ## colocar o puuid aqui

Vou pegar o histórico
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
time 100
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da partida
pegando detalhe da parti

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Dados salvos em: s3://partida-para-prever/team_100/match_history.json
Dados salvos em: s3://partida-para-prever/team_200/match_history.json
